In [ ]:
import os
import sys
import copy
import pickle
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from leap_ec.problem import FunctionProblem
import numpy as np
import numpy.linalg as nla
import scipy as sp
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [ ]:
from pyhms.config import TreeConfig, CMALevelConfig, EALevelConfig
from pyhms.tree import DemeTree
from pyhms.demes.single_pop_eas.sea import SEA
from pyhms.sprout.sprout_mechanisms import SproutMechanism, get_simple_sprout
from pyhms.sprout.sprout_filters import NBC_FarEnough, DemeLimit, LevelLimit
from pyhms.sprout.sprout_generators import NBC_Generator
from pyhms.stop_conditions.usc import dont_stop, metaepoch_limit
from pyhms.utils.misc_util import compute_centroid

In [ ]:
from mmop.DDRB import DDRB
import numpy as np 

---------

In [ ]:
# Common
bounds_base = [(-5.0, 5.0)]
dimensionality = 10

gsc = metaepoch_limit(40)
lvl_1_generations = 2
lvl_2_generations = 10
pop_size = 100
mutation_std = 2.0
metaepoch_lim_1=20
metaepoch_lim_2=20
sigma0 = 0.25

# NBC sprout
nbc_sprout = SproutMechanism(NBC_Generator(2.0, 0.8), [NBC_FarEnough(1.0, 2), DemeLimit(1)], [LevelLimit(20)])

# Simple sprout
simple_sprout = get_simple_sprout(5.0, 20)

In [ ]:
def setup_hms(function_problem, bounds, sprout, seed):
    options = {"random_state": seed}

    config = [
    EALevelConfig(
        ea_class=SEA, 
        generations=lvl_1_generations, 
        problem=function_problem, 
        bounds=bounds, 
        pop_size=pop_size,
        mutation_std=mutation_std,
        lsc=metaepoch_limit(metaepoch_lim_1)
        ),
    CMALevelConfig(
        problem=function_problem, 
        bounds=bounds,
        lsc=metaepoch_limit(metaepoch_lim_2),
        sigma0=sigma0,
        generations=lvl_2_generations
        )
    ]

    config = TreeConfig(config, gsc, sprout, options)
    return DemeTree(config)

def setup_nbc_hms(function_problem, bounds, sprout, seed):
    options = {"random_state": seed}

    config = [
    EALevelConfig(
        ea_class=SEA, 
        generations=lvl_1_generations, 
        problem=function_problem, 
        bounds=bounds, 
        pop_size=pop_size,
        mutation_std=mutation_std,
        lsc=metaepoch_limit(metaepoch_lim_1)
        ),
    CMALevelConfig(
        problem=function_problem, 
        bounds=bounds,
        lsc=metaepoch_limit(metaepoch_lim_2),
        sigma0=sigma0,
        generations=lvl_2_generations
        )
    ]

    config = TreeConfig(config, gsc, sprout, options)
    return DemeTree(config)

In [ ]:
testbed = [DDRB(i, 0) for i in range(1,11)]
for bench in testbed:
    bench.calc_problem_data()
    print(bench.statData.numGlobMin)

### Testing different parameter values

##### Changing simple sprout far enough during tests

In [ ]:
import dill as pkl

In [ ]:
far_enough = np.linspace(0.1, 10, 26)

bench_deme_numbers = []
bench_coverages = []
bench_optima_numbers = []

for bench_i in range(len(testbed)):
    print(f"Running problem {bench_i}")
    avg_deme_numbers = []
    avg_coverages = []
    for i in far_enough:
        deme_numbers = []
        coverages = []
        for k in range(10):
            bench_c = copy.deepcopy(testbed[bench_i])
            simple_sprout = get_simple_sprout(i, 20)
            hms_tree = setup_hms(FunctionProblem(bench_c.func_eval, maximize=False), bounds_base*dimensionality, simple_sprout, seed=bench_i+k)
            while not hms_tree._gsc(hms_tree):
                hms_tree.metaepoch_count += 1
                hms_tree.run_metaepoch()
                if not hms_tree._gsc(hms_tree):
                    hms_tree.run_sprout()

            deme_numbers.append(len(hms_tree.all_demes) - 1)
            covered = 0
            for j in range(bench_c.statData.numGlobMin):
                for _, deme in hms_tree.all_demes:
                    if nla.norm(deme.best_current_individual.genome - bench_c.statData.globMinima[j]) < bench_c.statData.nichRad[j]:
                        covered += 1
                        break
            coverages.append(covered)

        avg_deme_numbers.append(np.mean(deme_numbers))
        avg_coverages.append(np.mean(coverages))
    bench_deme_numbers.append(avg_deme_numbers)
    bench_coverages.append(avg_coverages)
    bench_optima_numbers.append(bench_c.statData.numGlobMin)

pkl.dump((far_enough, bench_deme_numbers, bench_coverages, bench_optima_numbers), open("simple_sprout.pkl", "wb"))

In [ ]:
nbc_far_enough = np.linspace(1.0, 3.0, 21)

bench_deme_numbers = []
bench_coverages = []
bench_optima_numbers = []

for bench_i in range(len(testbed)):
    print(f"Running problem {bench_i}")
    avg_deme_numbers = []
    avg_coverages = []
    for i in nbc_far_enough:
        deme_numbers = []
        coverages = []
        for k in range(10):
            bench_c = copy.deepcopy(testbed[bench_i])
            nbc_sprout = SproutMechanism(NBC_Generator(1.0, 0.8), [NBC_FarEnough(i, 2), DemeLimit(1)], [LevelLimit(20)])
            hms_tree = setup_hms(FunctionProblem(bench_c.func_eval, maximize=False), bounds_base*dimensionality, nbc_sprout, seed=bench_i+k)
            while not hms_tree._gsc(hms_tree):
                hms_tree.metaepoch_count += 1
                hms_tree.run_metaepoch()
                if not hms_tree._gsc(hms_tree):
                    hms_tree.run_sprout()

            deme_numbers.append(len(hms_tree.all_demes) - 1)
            covered = 0
            for j in range(bench_c.statData.numGlobMin):
                for _, deme in hms_tree.all_demes:
                    if nla.norm(deme.best_current_individual.genome - bench_c.statData.globMinima[j]) < bench_c.statData.nichRad[j]:
                        covered += 1
                        break
            coverages.append(covered)

        avg_deme_numbers.append(np.mean(deme_numbers))
        avg_coverages.append(np.mean(coverages))
    bench_deme_numbers.append(avg_deme_numbers)
    bench_coverages.append(avg_coverages)
    bench_optima_numbers.append(bench_c.statData.numGlobMin)

pkl.dump((nbc_far_enough, bench_deme_numbers, bench_coverages, bench_optima_numbers), open("nbc_far_enough.pkl", "wb"))

In [ ]:
nbc_cut = np.linspace(1.0, 3.0, 21)

bench_deme_numbers = []
bench_coverages = []
bench_optima_numbers = []

for bench_i in range(len(testbed)):
    print(f"Running problem {bench_i}")
    avg_deme_numbers = []
    avg_coverages = []
    for i in nbc_cut:
        deme_numbers = []
        coverages = []
        for k in range(10):
            bench_c = copy.deepcopy(testbed[bench_i])
            nbc_sprout = SproutMechanism(NBC_Generator(i, 0.8), [NBC_FarEnough(1.0, 2), DemeLimit(1)], [LevelLimit(20)])
            hms_tree = setup_hms(FunctionProblem(bench_c.func_eval, maximize=False), bounds_base*dimensionality, nbc_sprout, seed=bench_i+k)
            while not hms_tree._gsc(hms_tree):
                hms_tree.metaepoch_count += 1
                hms_tree.run_metaepoch()
                if not hms_tree._gsc(hms_tree):
                    hms_tree.run_sprout()

            deme_numbers.append(len(hms_tree.all_demes) - 1)
            covered = 0
            for j in range(bench_c.statData.numGlobMin):
                for _, deme in hms_tree.all_demes:
                    if nla.norm(deme.best_current_individual.genome - bench_c.statData.globMinima[j]) < bench_c.statData.nichRad[j]:
                        covered += 1
                        break
            coverages.append(covered)

        avg_deme_numbers.append(np.mean(deme_numbers))
        avg_coverages.append(np.mean(coverages))
    bench_deme_numbers.append(avg_deme_numbers)
    bench_coverages.append(avg_coverages)
    bench_optima_numbers.append(bench_c.statData.numGlobMin)

pkl.dump((nbc_cut, bench_deme_numbers, bench_coverages, bench_optima_numbers), open("nbc_cut.pkl", "wb"))

In [ ]:
nbc_trunc = np.linspace(0.1, 0.91, 17)

bench_deme_numbers = []
bench_coverages = []
bench_optima_numbers = []

for bench_i in range(len(testbed)):
    print(f"Running problem {bench_i}")
    avg_deme_numbers = []
    avg_coverages = []
    for i in nbc_trunc:
        deme_numbers = []
        coverages = []
        for k in range(10):
            bench_c = copy.deepcopy(testbed[bench_i])
            nbc_sprout = SproutMechanism(NBC_Generator(1.5, i), [NBC_FarEnough(1.5, 2), DemeLimit(1)], [LevelLimit(20)])
            hms_tree = setup_hms(FunctionProblem(bench_c.func_eval, maximize=False), bounds_base*dimensionality, nbc_sprout, seed=bench_i*10+k)
            while not hms_tree._gsc(hms_tree):
                hms_tree.metaepoch_count += 1
                hms_tree.run_metaepoch()
                if not hms_tree._gsc(hms_tree):
                    hms_tree.run_sprout()

            deme_numbers.append(len(hms_tree.all_demes) - 1)
            covered = 0
            for j in range(bench_c.statData.numGlobMin):
                for _, deme in hms_tree.all_demes:
                    if nla.norm(deme.best_current_individual.genome - bench_c.statData.globMinima[j]) < bench_c.statData.nichRad[j]:
                        covered += 1
                        break
            coverages.append(covered)

        avg_deme_numbers.append(np.mean(deme_numbers))
        avg_coverages.append(np.mean(coverages))
    bench_deme_numbers.append(avg_deme_numbers)
    bench_coverages.append(avg_coverages)
    bench_optima_numbers.append(bench_c.statData.numGlobMin)

pkl.dump((nbc_trunc, bench_deme_numbers, bench_coverages, bench_optima_numbers), open("nbc_trunc.pkl", "wb"))

--------------
### Coverage v budget test

In [ ]:
# NBC sprout
nbc_sprout = SproutMechanism(NBC_Generator(1.5, 0.8), [NBC_FarEnough(1.0, 2), DemeLimit(1)], [LevelLimit(20)])

# Simple sprout
simple_sprout = get_simple_sprout(4.0, 20)

algos = [(setup_hms, simple_sprout), (setup_nbc_hms, nbc_sprout)]

In [ ]:
results = {}
for algo in algos:
    results[algo[1]] = []
    for bench in testbed:
        avg_coverage = []
        avg_budget = []
        for i in range(50):
            bench_c = copy.deepcopy(bench)
            hms_tree = algo[0](FunctionProblem(bench_c.func_eval, maximize=False), bounds_base*dimensionality, algo[1], seed=i+1)
            while not hms_tree._gsc(hms_tree):
                hms_tree.metaepoch_count += 1
                hms_tree.run_metaepoch()
                if not hms_tree._gsc(hms_tree):
                    hms_tree.run_sprout()
            covered = 0
            for j in range(bench_c.statData.numGlobMin):
                for _, deme in hms_tree.all_demes:
                    if nla.norm(deme.best_current_individual.genome - bench_c.statData.globMinima[j]) < bench_c.statData.nichRad[j]:
                        covered += 1
                        break
            budget = 0
            for _, deme in hms_tree.all_demes:
                budget += sum([1 for pop in deme.history for _ in pop])
            avg_coverage.append(covered)
            avg_budget.append(budget)
        results[algo[1]].append((avg_coverage, avg_budget))
with open("cov_results.pkl", "wb") as f:
    pkl.dump(results, f)